<a href="https://colab.research.google.com/github/riphunter7001x/RAG/blob/main/MergerRetriever_and_LongContextReorder_with_LLama3%26chromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the Require Libraries

In [3]:
!pip install -qU langchain chromadb huggingface_hub sentence-transformers pypdf openai tiktoken

In [4]:
!pip install -U langchain-community

# Let's Load the Data Now...

In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
loader_harrypotter  = PyPDFLoader("/content/hpo(8).pdf")

In [8]:
documnet_harrypotter = loader_harrypotter.load()

In [9]:
print(len(documnet_harrypotter))

250


In [10]:
loader_got = PyPDFLoader("/content/scf(12).pdf")

In [11]:
documnet_got = loader_got.load()

In [12]:
print(len(documnet_got))


577


# Let's create the text splitter for chunking

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [15]:
text_harrypotter = text_splitter.split_documents(documnet_harrypotter)

In [16]:
text_got = text_splitter.split_documents(documnet_got)

In [17]:
print(len(text_harrypotter))

1189


In [18]:
print(len(text_got))

4232


# Load the Embedding Model to Conver the Data into Vector

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings,HuggingFaceBgeEmbeddings

In [21]:
# hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [20]:
hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
# os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [ ]:
# openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Now ingest the Data into the Chroma Database

In [22]:
from langchain.vectorstores import Chroma
import chromadb

In [23]:
import os
os.getcwd()

'/content'

In [24]:
CURRENT_DIR = os.path.dirname(os.path.abspath("."))

In [25]:
CURRENT_DIR

'/'

In [26]:
DB_DIR = os.path.join(CURRENT_DIR, "/content/db")

In [27]:
DB_DIR

'/content/db'

In [28]:
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [29]:
harrypotter_vectorstore = Chroma.from_documents(text_harrypotter,
                                       hf_bge_embeddings,
                                       client_settings=client_settings,
                                       collection_name="harrypotter",
                                       collection_metadata={"hnsw":"cosine"},
                                       persist_directory="/store/harrypotter")

In [30]:
got_vectorstore = Chroma.from_documents(text_got,
                                       hf_bge_embeddings,
                                       client_settings=client_settings,
                                       collection_name="got",
                                       collection_metadata={"hnsw":"cosine"},
                                       persist_directory="/store/got")

 # Now Crearte a Retriever

In [31]:
retriever_harrypotter = harrypotter_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True}) #for mumtimodal retrival

In [32]:
retriever_got = got_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

# Let's Merge both Retriever

# It is also called lord of retriever(LOTR)

In [33]:
from langchain.retrievers.merger_retriever import MergerRetriever

In [34]:
lotr = MergerRetriever(retrievers=[retriever_harrypotter, retriever_got])

In [35]:
for chunks in lotr.get_relevant_documents("Who was the jon snow?"):
    print(chunks.page_content)

155itself covered in several feet of snow. The lake froze solid and the
Weasley twins were punished for bewitching several snowballs so thatthey followed Quirrell around, bouncing off the back of his turban. Thefew owls that managed to battle their way through the stormy sky todeliver mail had to be nursed back to health by Hagrid before they couldfly off again.
No one could wait for the holidays to start. While the Gryffindor common
was done with denials. He was who he was; Jon Snow, bastard and oathbreaker, motherless,
friendless, and damned. For the rest of his life—however long that might be—he would be
condemned to be an outsider, the silent man standing in the shadows who dares not speak his true
name. Wherever he might go throughout the Seven Kingdoms, he would need to live a lie, lest every
man’s hand be raised against him. But it made no matter, so long as he lived long enough to take his
"Some say he died. Codswallop, in my opinion. Dunno if he had enough
human left in him to

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [36]:
for chunks in lotr.get_relevant_documents("Who is a harry potter?"):
    print(chunks.page_content)

Harry turned over his card and read:ALBUS DUMBLEDORE
CURRENTLY HEADMASTER OF HOGWARTS
Considered by many the greatest wizard of modern times, Dumbledore is
particularly famous for his defeat of the dark wizard Grindelwald in1945, for the discovery of the twelve uses of dragon's blood, and hiswork on alchemy with his partner, Nicolas Flamel. Professor Dumbledore
enjoys chamber music and tenpin bowling.
Harry turned the card back over and saw, to his astonishment, that
crimson field. The Lannister words are 
Hear Me Roar!
TYWIN LANNISTER, Lord of Casterly Rock, Warden of the West, Shield of Lannisport,
—his wife, {LADY JOANNA}, a cousin, died in childbed,
—their children:
—SER JAIME, called the Kingslayer, heir to Casterly Rock, a twin to Cersei,
—QUEEN CERSEI, wife of King Robert I Baratheon, a twin to Jaime,
—TYRION, called the Imp, a dwarf,
—his siblings:
—SER KEVAN, his eldest brother,
—his wife, DORNA, of House Swyft,
he is?"
"Who?""Harry Potter!"
Harry heard the little girl's voice

## See this result is too much messy now lets refine it according to the question and overcome the situation of lost in middle

# Now After understanding step by step it create a pipeline for LLM

In [37]:
from langchain.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder

In [38]:
from re import search
filter = EmbeddingsRedundantFilter(embeddings=hf_bge_embeddings)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr,search_kwargs={"k": 3, "include_metadata": True}
)


In [39]:
docs = compression_retriever_reordered.get_relevant_documents("What is esops?")
print(len(docs))
#
print(docs[0].page_content)

10
made for good climbing. You could go straight up to where the gargoyles leaned out blindly over
empty space, and swing from gargoyle to gargoyle, hand over hand, around to the north side. From
there, if you really stretched, you could reach out and pull yourself over to the broken tower where it
leaned close. The last part was the scramble up the blackened stones to the eyrie, no more than ten
feet, and then the crows would come round to see if you’d brought any corn.


# Load the model from huggingface

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=3705041 sha256=3ebf6e4539f32e88738d5fcc3a38b6d6f2eb76f6db9eaa7abbcc9d0fdd1588b0
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python


In [ ]:
from langchain.llms import LlamaCpp
llms = LlamaCpp(streaming=True,
                   model_path="/content/drive/MyDrive/zephyr-7b-beta.Q4_K_M.gguf",
                   max_tokens = 1500,
                   temperature=0.75,
                   top_p=1,
                   gpu_layers=0,
                   stream=True,
                   verbose=True,n_threads = int(os.cpu_count()/2),
                   n_ctx=4096)

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! gpu_layers is not default parameter.
                gpu_layers was transferred to model_kwargs.
                Please confirm that gpu_layers is what you intended.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggi

In [43]:
!pip install langchain_groq

from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
from langchain_groq import ChatGroq


In [45]:
llms = ChatGroq(
    model="llama3-70b-8192"
)


In [46]:
from langchain.chains import RetrievalQA

In [47]:
qa = RetrievalQA.from_chain_type(
      llm=llms,
      chain_type="stuff",
      retriever = compression_retriever_reordered,
      return_source_documents = True
)

In [50]:
query ="who is jon snow?"
results = qa.invoke(query)
print(results['result'])
#
print(results["source_documents"])

Jon Snow is a character from the A Song of Ice and Fire book series by George R.R. Martin, and its TV adaptation, Game of Thrones. He is a bastard son of Ned Stark, the Warden of the North, and is one of the main protagonists of the series.
[_DocumentWithState(page_content='broken toy, half-embedded in the ice.\nOn the far side of the catapult, a muffled voice called out a challenge. “Who goes there? Halt!”\nTyrion stopped. “If I halt too long I’ll freeze in place, Jon,” he said as a shaggy pale shape slid\ntoward him silently and sniffed at his furs. “Hello, Ghost.”\nJon Snow moved closer. He looked bigger and heavier in his layers of fur and leather, the hood\nof his cloak pulled down over his face. “Lannister,” he said, yanking loose the scarf to uncover his', metadata={'page': 151, 'source': '/content/scf(12).pdf'}, state={'embedded_doc': [0.005699861329048872, -0.00784630049020052, -0.006249286234378815, -0.0065004569478333, -0.023128587752580643, -0.04068280756473541, -0.04016769

In [51]:
print(results['result'])

Jon Snow is a character from the A Song of Ice and Fire book series by George R.R. Martin, and its TV adaptation, Game of Thrones. He is a bastard son of Ned Stark, the Warden of the North, and is one of the main protagonists of the series.



 Jon Snow is a character in George R.R. Martin's "A Song of Ice and Fire" series, which has been adapted into the popular HBO show "Game of Thrones." In the context provided, he is described as Bran Stark's bastard brother and is mentioned as moving closer to Bran during a scene in which Bran witnesses his father, King Robert Baratheon, sentencing Ned Stark to death for treason. Jon Snow has been portrayed by actor Kit Harington on the TV show "Game of Thrones."

In [52]:
results = qa("who is a harry potter?")
print(results['result'])
#
print(results["source_documents"])
#
for source in  results["source_documents"]:
    print(source.metadata)

Harry Potter is the main protagonist in J.K. Rowling's series of fantasy novels. He is a young wizard who attends Hogwarts School of Witchcraft and Wizardry.
[_DocumentWithState(page_content='crimson field. The Lannister words are \nHear Me Roar!\nTYWIN LANNISTER, Lord of Casterly Rock, Warden of the West, Shield of Lannisport,\n—his wife, {LADY JOANNA}, a cousin, died in childbed,\n—their children:\n—SER JAIME, called the Kingslayer, heir to Casterly Rock, a twin to Cersei,\n—QUEEN CERSEI, wife of King Robert I Baratheon, a twin to Jaime,\n—TYRION, called the Imp, a dwarf,\n—his siblings:\n—SER KEVAN, his eldest brother,\n—his wife, DORNA, of House Swyft,', metadata={'page': 565, 'source': '/content/scf(12).pdf'}, state={'embedded_doc': [0.0010452455608174205, -0.009499212726950645, 0.009500144980847836, -0.005465999711304903, -0.004582940135151148, -0.0236272681504488, -0.030875854194164276, 0.007241127081215382, 0.01744363084435463, -0.017344338819384575, 0.018973909318447113, -0.00

Harry Potter is the main character in J.K. Rowling's series of novels and films about magic and wizards. He is an orphan who discovers that he has magical powers and goes to attend Hogwarts School of Witchcraft and Wizardry, where he makes friends and battles evil forces like Lord Voldemort.

Harry Potter is the main protagonist in J.K. Rowling's series of fantasy novels. He is a young wizard who attends Hogwarts School of Witchcraft and Wizardry.

In [53]:
results = qa.invoke("How does Jon Snow's relationship with the Stark family influence his identity and decisions throughout A Game of Thrones?")


In [54]:
results

{'query': "How does Jon Snow's relationship with the Stark family influence his identity and decisions throughout A Game of Thrones?",
 'result': "Jon Snow's relationship with the Stark family has a significant impact on his identity and decisions throughout A Game of Thrones.\n\nAs a bastard son, Jon Snow is not fully part of the Stark family, but he is still closely tied to them. He is raised at Winterfell alongside Robb, Sansa, Arya, and Bran, and is treated with kindness and respect by Eddard Stark, the Warden of the North. However, Jon's bastardy and his mother's unknown identity create a sense of insecurity and uncertainty about his place in the family.\n\nThis ambiguity affects Jon's sense of belonging and identity. He feels like an outsider, not fully part of the Stark family, but not fully apart from them either. This feeling of not quite belonging is a recurring theme throughout the book.\n\nJon's relationship with the Stark family also influences his decisions. He is deeply 

In [55]:
print(results['result'])



Jon Snow's relationship with the Stark family has a significant impact on his identity and decisions throughout A Game of Thrones.

As a bastard son, Jon Snow is not fully part of the Stark family, but he is still closely tied to them. He is raised at Winterfell alongside Robb, Sansa, Arya, and Bran, and is treated with kindness and respect by Eddard Stark, the Warden of the North. However, Jon's bastardy and his mother's unknown identity create a sense of insecurity and uncertainty about his place in the family.

This ambiguity affects Jon's sense of belonging and identity. He feels like an outsider, not fully part of the Stark family, but not fully apart from them either. This feeling of not quite belonging is a recurring theme throughout the book.

Jon's relationship with the Stark family also influences his decisions. He is deeply loyal to the Starks, particularly to his half-brother Robb, and feels a strong sense of duty to protect and serve them. This loyalty motivates him to joi

In [56]:
print(results["source_documents"])

[_DocumentWithState(page_content='Bran saw his father’s face change, saw the other men exchange glances. He loved Jon with all his\nheart at that moment. Even at seven, Bran understood what his brother had done. The count had come\nright only because Jon had omitted himself. He had included the girls, included even Rickon, the\nbaby, but not the bastard who bore the surname Snow, the name that custom decreed be given to all\nthose in the north unlucky enough to be born with no name of their own.', metadata={'page': 19, 'source': '/content/scf(12).pdf'}, state={'embedded_doc': [-0.013021159917116165, 0.005394859705120325, 0.002666027285158634, -0.0042250570841133595, -0.02116803638637066, -0.026622168719768524, -0.04192778095602989, 0.01825118064880371, 0.009620786644518375, 0.014816517941653728, 0.05824019014835358, 0.006234593223780394, 0.014605659060180187, -0.013230429962277412, -0.03204381838440895, -0.0015016727847978473, -0.051746007055044174, -0.033957529813051224, -0.0182796083

In [57]:
for source in  results["source_documents"]:
    print(source.metadata)

{'page': 19, 'source': '/content/scf(12).pdf'}
{'page': 191, 'source': '/content/scf(12).pdf'}
{'page': 92, 'source': '/content/scf(12).pdf'}
{'page': 537, 'source': '/content/scf(12).pdf'}
{'page': 152, 'source': '/content/scf(12).pdf'}
{'page': 99, 'source': '/content/hpo(8).pdf'}
{'page': 212, 'source': '/content/hpo(8).pdf'}
{'page': 18, 'source': '/content/hpo(8).pdf'}
{'page': 163, 'source': '/content/hpo(8).pdf'}
{'page': 22, 'source': '/content/hpo(8).pdf'}
